In [56]:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [57]:
sc = spark.sparkContext

## Spark SQL

* En Spark 1.6 se introdujo una nueva abstracción de programación llamada API estructurada. Esta es la forma preferida para realizar el procesamiento de datos en la mayoría de los casos de uso.

* En esta nueva forma de hacer el procesamiento de datos, los datos deben organizarse en un formato estructurado y la lógica de cálculo de datos debe seguir una determinada estructura. Con estas dos piezas de información, Spark puede realizar optimizaciones para acelerar las aplicaciones de procesamiento de datos.

* El componente SparkSQL está construido sobre el viejo y confiable componente SparkCore. Esta arquitectura en capas significa que cualquier mejora en el componente Spark Core estará disponible automáticamente para el componente para SQL.

* El concepto DF se inspiró en el concepto de pandasDF de python. La principal diferencia es que un DF en Spark puede manejar un gran volumen de datos que se distribuyen en muchas máquinas.

* Un concepto fundamental que diferencia a los datos estructurados de los no estructurados es un **esquema** que define la estructura de los datos en forma de nombres de columna y tipos de datos asociados. El concepto de **esquema** es una parte integral de las APIs estructuradas de Spark.

* Los datos estructurados a menudo se capturan en un formato determinado. Algunos de estos formatos están basados en textos y algunos de ellos están basados en binario.

    * Los formatos comunes para datos de texto son CSV, XML y JSON 
    * Los formatos comunes para datos binarios son agro, parquet y ORC.
<br>.
* El módulo SparkSQL y facilita la lectura y escritura de datos desde y hacia cualquiera de estos formatos. Un beneficio inesperado que surge de esta versatilidad es que Spark se puede utilizar como una herramienta de conversión de formato de datos.

### Crear un DF a partir de un RDD

Hay muchas formas de crear un DF, pero **siempre hay que proporcionar un esquema**, ya sea implícita o explícitamente.

In [58]:
rdd = sc.parallelize([item for item in range(10)]).map(lambda x: (x, x ** 2))

rdd.collect()

[(0, 0),
 (1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (5, 25),
 (6, 36),
 (7, 49),
 (8, 64),
 (9, 81)]

In [59]:
# Para crear un DF a parti de un RDD no tenemos más que pasarle a la función .toDF los nombres de las columnas como lista
df = rdd.toDF(['numero', 'cuadrado'])

# Podemos ver el esquema de los datos, que nos informa del tipo de datos y de si acepta nulos o no
df.printSchema()

# Podemos ver el DF
df.show()

root
 |-- numero: long (nullable = true)
 |-- cuadrado: long (nullable = true)

+------+--------+
|numero|cuadrado|
+------+--------+
|     0|       0|
|     1|       1|
|     2|       4|
|     3|       9|
|     4|      16|
|     5|      25|
|     6|      36|
|     7|      49|
|     8|      64|
|     9|      81|
+------+--------+



##### También podemos crear un DF a partir de un RDD definiendo nosotros el esquema

In [60]:
rdd1 = sc.parallelize([(1, 'Jose', 35.5), (2, 'Teresa', 54.3), (3, 'Katia', 12.7)])

In [61]:
# Método 1: creamos el esquema a partir de las classes de pyspark.sql.types

esquema1 = StructType(
    [
     StructField('id', IntegerType(), True),
     StructField('nombre', StringType(), True),
     StructField('saldo', DoubleType(), True)
    ]
)

In [62]:
# Método 2: creamos el esquema a partir de un string

esquema2 = "`id` INT, `nombre` STRING, `saldo` DOUBLE"

In [63]:
# Finalmente, para crear un DF aplicando nuestro esquema, usamos la función .createDtaFrame, pasándole el RDD y el esquema

df1 = spark.createDataFrame(rdd1, schema=esquema1)

df1.printSchema()

df1.show()

root
 |-- id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- saldo: double (nullable = true)

+---+------+-----+
| id|nombre|saldo|
+---+------+-----+
|  1|  Jose| 35.5|
|  2|Teresa| 54.3|
|  3| Katia| 12.7|
+---+------+-----+



In [64]:
df1 = spark.createDataFrame(rdd1, schema=esquema2)

df1.printSchema()

df1.show()

root
 |-- id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- saldo: double (nullable = true)

+---+------+-----+
| id|nombre|saldo|
+---+------+-----+
|  1|  Jose| 35.5|
|  2|Teresa| 54.3|
|  3| Katia| 12.7|
+---+------+-----+



### Crear un DF a partir de fuentes de datos

Las dos clases principales en SparkSQL para leer y escribir datos son DataFrameReader y DataFrameWriter respectivamente.

#### DataFrameReader

* Una instancia de la clase DataFrameReader está disponible como read en la sesión de Spark y la podemos invocar a través de **spark.read**

* El patrón común para interactuar con DataFrameReader es **spark.read.format(...).option('key', 'value').schema(...).load()**

    * **.format(...)** no es opcional. Puede ser una de las fuente de datos integradas o un formato de dato personalizado

        * **Formato integrado:** se puede usar un nombre corto como por ejemplo json, parquet, jdbc, orc, csv, text...
        * **Formato personalizado:** debe proporcionar un nombre completo
<br>.
    * **.option('key', 'value')** es opcional porque DataFrameReader tiene un conjunto de opciones predeterminadas para cada formato de fuente de datos. Podemos anular estos valores predeterminados proporcionando un valor a la función option('key', 'value')

    * **.schema(...)** puede ser opcional porque algunas fuentes de datos tienen el esquema incrustado dentro de los archivos de datos, podemos pensar en .parquet u .orc En estos casos el esquema se infiere automáticamente, para otros casos es posible que deba proporcionar un esquema.

* Para leer los datos hay dos alternativas aplicables a todos los formatos:

    * **spark.read.<extension>("<path>")** por ejemplo, spark.read.json("/path/to/file.json")
    * **spark.read.format("<extension>").load("<path>")** por ejemplo spark.read.format("json").load("/path/to/file.json")



In [65]:
# Crear un DF a partir de un archivo de texto
dftxt = spark.read.text('./data/data_DF/dataTXT.txt')

dftxt.show(truncate=False)

+-----------------------------------------------------------------------+
|value                                                                  |
+-----------------------------------------------------------------------+
|Estamos en el curso de pyspark                                         |
|En este capítulo estamos estudiando el API SQL de Saprk                |
|En esta sección estamos creado dataframes a partir de fuentes de datos,|
|y en este ejemplo creamos un dataframe a partir de un texto plano      |
+-----------------------------------------------------------------------+



In [66]:
# Crear un DataFrame mediante la lectura de un archivo csv

dfcsv = spark.read.csv('./data/data_DF/dataCSV.csv')

# Aquí mete el nombre de las columnas en la primera fila y da nombres nuevos predeterminados
dfcsv.show(n=10, truncate=False, vertical=False)

+-----------+-------------+-----------------------------------------------------------------+---------------------+-----------+------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [67]:
# Para que tome la primera fila como nombre de columnas:
dfcsv1 = spark.read.option('header', 'true').csv('./data/data_DF/dataCSV.csv')

dfcsv1.show(n=10, truncate=False, vertical=False)

+-----------+-------------+-----------------------------------------------------------------+---------------------+-----------+------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [68]:
# Leer un archivo de texto con un delimitador diferente

dftxt1 = spark.read.option('header', 'true').option('delimiter', '|').csv('./data/data_DF/dataTab.txt')

dftxt1.show()


+----+----+----------+-----+
|pais|edad|     fecha|color|
+----+----+----------+-----+
|  MX|  23|2021-02-21| rojo|
|  CA|  56|2021-06-10| azul|
|  US|  32|2020-06-02|verde|
+----+----+----------+-----+



In [69]:
# Crear un DataFrame a partir de un json proporcionando un schema
json_schema =  StructType(
    [
     StructField('color', StringType(), True),
     StructField('edad', IntegerType(), True),
     StructField('fecha', DateType(), True),
     StructField('pais', StringType(), True)
    ]
)

dfjson = spark.read.schema(json_schema).json('./data/data_DF/dataJSON.json')

dfjson.show()

dfjson.printSchema()

+-----+----+----------+----+
|color|edad|     fecha|pais|
+-----+----+----------+----+
| rojo|null|2021-02-21|  MX|
| azul|null|2021-06-10|  CA|
|verde|null|2020-06-02|  US|
+-----+----+----------+----+

root
 |-- color: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- fecha: date (nullable = true)
 |-- pais: string (nullable = true)



In [70]:
# Crear un DataFrame a partir de un archivo parquet
dfparquet = spark.read.parquet('./data/data_DF/dataPARQUET.parquet')

dfparquet.show(n=10, truncate=False, vertical=False)

+-----------+-------------+-----------------------------------------------------------------+---------------------+-----------+------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [71]:
# Otra alternativa para leer desde una fuente de datos parquet en este caso
dfparquet1 = spark.read.format('parquet').load('./data/data_DF/dataPARQUET.parquet')

dfparquet1.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)

